# Five lessons from the Advent of Code

### Eugene Ha

<p>
PyData Berlin<br>
16 Jan 2019

## [Advent of Code](https://adventofcode.com)

- By [Eric Wastl](http://was.tl) — a creative _tour de force_

- 25 (mischievous) programming challenges, covering spectrum of CS topics

- Each problem has 2 parts, revealed one at a time

- 2nd part often changes "business requirements"—refactor or take new approach

## Some lessons learned

## _1_

## Use [Numba](https://numba.pydata.org) to speed up your code for free\*

\* Conditions and restrictions apply. See your 🐍 guru.

In [1]:
## NO SLIDE
import numpy as np

def make_input(n, pos, init):
    pos = np.array(pos, dtype='int')
    scores = np.ones(int(n + 10), dtype='int')
    scores[:len(init)] = init
    return pos, scores, len(init)

pos, scores, n = make_input(580741, (0, 1), (3, 7))

### An ordinary function ([Day 14](https://adventofcode.com/2018/day/14))

In [2]:
import numpy as np

def fill_scores(pos, scores, n):
    scores = scores.copy()
    while n < len(scores):
        s = np.sum(scores[pos])
        if s >= 10:
            scores[n + 1] = s - 10
            n += 2
        else:
            scores[n] = s
            n += 1
        pos = (pos + 1 + scores[pos]) % n
    return scores

In [3]:
%%timeit
fill_scores(pos, scores, n)

3.28 s ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


🐢

### Same code JIT-compiled

In [4]:
from numba import jit

@jit(nopython=True)
def fill_scores_jit(pos, scores, n):
    scores = scores.copy()
    while n < len(scores):
        s = np.sum(scores[pos])
        if s >= 10:
            scores[n + 1] = s - 10
            n += 2
        else:
            scores[n] = s
            n += 1
        pos = (pos + 1 + scores[pos]) % n
    return scores

# JIT-compiled on first call
fill_scores_jit(pos, scores, n);

In [5]:
%%timeit
fill_scores_jit(pos, scores, n)

194 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**>10x** faster 🐰

## _2_

## Study APL to learn effective [NumPy](https://numpy.org)

### A taste of APL

Game of Life (cellular automaton)

```APL
life←{↑1 ⍵∨.∧3 4=+/,¯1 0 1∘.⊖¯1 0 1∘.⌽⊂⍵}
```

<img src="images/life.gif">

## Logging trees: a cellular automaton ([Day 18](https://adventofcode.com/2018/day/18))

```
.#.#...|#.    .......##.    .......#..       .||##.....
.....#|##|    ......|###    ......|#..       ||###.....
.|..|...#.    .|..|...#.    .|.|||....       ||##......
..|#.....#    ..|#||...#    ..##|||..#       |##.....##
#.#|||#|#|    ..##||.|#|    ..###|||#|       |##.....##
...#.||...    ...#||||..    ...#|||||.  ...  |##....##|
.|....|...    ||...|||..    |||||||||.       ||##.####|
||...#|.#|    |||||.||.|    ||||||||||       ||#####|||
|.||||..|.    ||||||||||    ||||||||||       ||||#|||||
...#.|..|.    ....||..|.    .|||||||||       ||||||||||

0             1             2                10
```

### Neighboring cells

In [6]:
import numpy as np

def neighbors(area):
    "Array of the 8 neighbor-matrices (3D-array)."
    amb, rows, cols = ambient_region(area.shape)
    amb[np.ix_(rows, cols)] = area
    shifts = [amb[np.ix_(rows + i, cols + j)] for i, j in SHIFTS]
    return np.array(shifts)

In [7]:
SHIFTS = (
    (-1, -1), (-1, 0), (-1, 1),
    ( 0, -1),          ( 0, 1),
    ( 1, -1), ( 1, 0), ( 1, 1),
)

In [8]:
def ambient_region(shape):
    shape = np.array(shape) + 2
    rows, cols = (np.arange(1, s - 1) for s in shape)
    amb = np.zeros(shape, dtype='int')
    return amb, rows, cols

### Cell-transition rules

  - `'.' → '|'`&nbsp;: &nbsp;when 3 or more neighbors are `'|'`
  
  - `'|' → '#'`&nbsp;: &nbsp;when 3 or more neighbors are `'#'`
  
  - `'#' → '.'`&nbsp;: &nbsp;when 0 neighbors are `'|'` or 0 neighbors are `'#'`

In [9]:
def rules(area):
    "Cell-transition rules in a given area (matrix)."
    nbhs = neighbors(area)
    cnts = [np.sum(nbhs == i, axis=0) for i in (1, 2)]
    return (
        (area == 0) & (cnts[0] >= 3),
        (area == 1) & (cnts[1] >= 3),
        (area == 2) & ((cnts[0] == 0) | (cnts[1] == 0)),
    )

### Logging trees

In [10]:
from functools import reduce
from operator  import or_

def log_trees(area):
    area = area.copy()
    transition = reduce(or_, rules(area))
    area[transition] = (area[transition] + 1) % 3
    return area

In [11]:
## NO SLIDE
def grid(scan, acre={a: i for i, a in enumerate('.|#')}):
    acres = [[acre[a] for a in line] for line in scan]
    return np.array(acres)

def draw(area, acre=np.array(list('.|#'))):
    grid = acre[area]
    for line in grid:
        print(''.join(line))
        
scan = (
    '.#.#...|#.',
    '.....#|##|',
    '.|..|...#.',
    '..|#.....#',
    '#.#|||#|#|',
    '...#.||...',
)
area = grid(scan)

In [12]:
draw(area)

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...


In [13]:
draw(log_trees(log_trees(area)))

.......#..
......|#..
.|.|||....
..##|||..#
..###|||#|
...#|||...


## _3_

## Write declarative code with the [Functional Triumvirate](https://docs.python.org/3/library/functional.html)

### The [Functional Triumvirate](https://docs.python.org/3/library/functional.html) (in the [Standard Library](https://docs.python.org/3/library/index.html))

  - [`itertools`](https://docs.python.org/3/library/itertools.html) (especially)

  - [`functools`](https://docs.python.org/3/library/functools.html)

  - [`operator`](https://docs.python.org/3/library/operator.html)

Python is [_not_ a functional programming language](http://python-history.blogspot.com/2009/04/origins-of-pythons-functional-features.html).

But you can still use functional idioms to write **declarative** code.

### Handy functional idioms

**Iterate** a function, indefinitely:

In [14]:
import itertools as it

def iterate(f, init):
    apply = lambda x, _: f(x)
    return it.accumulate(it.repeat(init), apply)

**Split** a sequence by key — no [🐼](https://pandas.pydata.org) required:

In [15]:
def split(xs, *, key):
    groups = it.groupby(sorted(xs, key=key), key=key)
    return tuple(tuple(g) for _, g in groups)

Return the **n-th item** (from [Itertools Recipes](https://docs.python.org/3/library/itertools.html#itertools-recipes)):

In [16]:
def nth(iterable, n, default=None):
    return next(it.islice(iterable, n, None), default)

### Back to logging trees ([Day 18](https://adventofcode.com/2018/day/18))

In [17]:
## NO SLIDE
scan = (
    '.#.#...|#.',
    '.....#|##|',
    '.|..|...#.',
    '..|#.....#',
    '#.#|||#|#|',
    '...#.||...',
    '.|....|...',
    '||...#|.#|',
    '|.||||..|.',
    '...#.|..|.',
)
area = grid(scan)

In [18]:
draw(area)

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.


In [19]:
area10 = nth(iterate(log_trees, area), 10)
draw(area10)

.||##.....
||###.....
||##......
|##.....##
|##.....##
|##....##|
||##.####|
||#####|||
||||#|||||
||||||||||


## _4_

## Maintain simplicity with [memoization](https://en.wikipedia.org/wiki/Memoization)

### Effectively refactor _without_ rewriting

  - If `area` shape doesn't change, neither does `ambient_region()`
  - But it keeps getting called by `neighbors()`!
  - Refactor so that `ambient_region()` is called _only_ when needed?

In [20]:
def neighbors(area):
    "Array of the 8 neighbor-matrices (3D-array)."
    amb, rows, cols = ambient_region(area.shape)
    amb[np.ix_(rows, cols)] = area
    shifts = [amb[np.ix_(rows + i, cols + j)] for i, j in SHIFTS]
    return np.array(shifts)

Instead of modifying `neighbors()` **and its caller**, make `ambient_region()` effectively **constant** by memoizing:

In [21]:
from functools import lru_cache

@lru_cache()
def ambient_region(shape):
    shape = np.array(shape) + 2
    rows, cols = (np.arange(1, s - 1) for s in shape)
    amb = np.zeros(shape, dtype='int')
    return amb, rows, cols

## _5_

## See your data with [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/)

In [22]:
## DEMO
with open('data/input-18', 'r') as f:
    scan = [line.rstrip() for line in list(f)]
area = grid(scan)

Take 1000 logging rounds:

In [23]:
## DEMO
logged_trees = list(it.islice(iterate(log_trees, area), 1000))

Page through the first 100:

In [24]:
## DEMO
from ipywidgets import interact, IntSlider

@interact(n=IntSlider(min=0, max=100, value=0))
def _(n): draw(logged_trees[n])

interactive(children=(IntSlider(value=0, description='n'), Output()), _dom_classes=('widget-interact',))

## The moral of the story

- Use [Numba](https://numba.pydata.org) to **speed up** your code for "free"

- Study APL to learn **effective [NumPy](https://numpy.org)**

- Write **declarative** code with the [Functional Triumvirate](https://docs.python.org/3/library/functional.html)

- Maintain **simplicity** with [memoization](https://en.wikipedia.org/wiki/Memoization)

- **See** your data with [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/)

## Thanks for your attention!

[github.com/egnha](https://github.com/egnha)<br>
[eugeneha.com](https://eugeneha.com)